<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 4 / SPRINT 2 / NOTE 1*

# 📝 Assignment

---


# Count-based_Representation

indeed.com 에서 Data Scientist 키워드로 Job descrition을 찾아 스크래핑한 데이터를 이용해 과제를 진행해 보겠습니다.

[Data_Scienties.csv](https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/indeed/Data_Scientist.csv) 파일에는 1300여개의 Data Scientist job description 정보가 담겨 있습니다.

## 1. 데이터 전처리 (Text preprocessing)

In [1]:
import re
import string

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 0) 텍스트 분석에 앞서 데이터 전처리를 진행합니다.

- 파일을 불러온 후 title, company, description 에 해당하는 Column만 남겨주세요.
- 중복값을 제거하세요.

In [2]:
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/indeed/Data_Scientist.csv')

In [8]:
df_0 = df[['title','company','description']].drop_duplicates()
df_0.head()

,title,company,description
0,Data Scientist (Structured Products),EquiTrust Life Insurance Company,Job Details\nDescription\nEssential Duties and...
2,"Specialist, Data Science",Nationwide,As a team member in the Finance and Internal A...
4,Sr. Data Scientist (Remote),American Credit Acceptance,Overview:\nAmerican Credit Acceptance seeks a ...
5,Data Scientist Associate Sr (DADS06) BTB - LEG...,"JPMorgan Chase Bank, N.A.",J.P. Morgan's Corporate & Investment Bank (CIB...
6,Data Scientist,VyStar Credit Union,"At VyStar, we offer competitive pay, an excell..."


In [12]:
df_0.shape

(757, 3)

### 1) 토큰을 정제합니다.

- 문자를 소문자로 통일
- 분석에 관련 없는 정보 제거
- 이번 과제는 `spacy` 로부터 `"en_core_web_sm"` 을 로드하여 진행해주세요.

- **문항 1) 대문자를 소문자로 변경하는 함수를 입력하세요.**
- **문항 2) 정규 표현식을 사용하여 re 라이브러리에서 알파벳 소문자, 숫자만 받을 수 있는 코드를 작성하세요.**

In [42]:
import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_sm")
tokenizer = Tokenizer(nlp.vocab)

tokens = []

for doc in tokenizer.pipe(df_0['description']):
  doc_tokens = [re.sub(r"[^a-z0-9]","",token.text.lower()) for token in doc]
  tokens.append(doc_tokens)

df_0['token'] = tokens
df_0['token'].tail(1)

1299    [tldr, , spring, is, accelerating, the, discov...
Name: token, dtype: object

### 2) 정제한 토큰을 시각화 합니다.

- Top 10 토큰을 프린트 합니다.
- 토큰의 수, 빈도 순위, 존재 문서 수, 비율 등 정보를 계산합니다.
- 토큰 순위에 따른 퍼센트 누적 분포 그래프를 시각화합니다.

- **문항 3) 추천 토큰 순위 10개 단어를 입력하세요.**

In [25]:
from collections import Counter

# Counter 객체는 리스트요소의 값과 요소의 갯수를 카운트 하여 저장하고 있습니다.
# 카운터 객체는 .update 메소드로 계속 업데이트 가능합니다.
word_counts = Counter()


In [26]:
def word_count(docs):
    """
    토큰화된 문서들을 입력받아 토큰별로 개수를 카운트 하고 관련된 속성을 가진 데이터프레임을 반환합니다.
    
    Args:
        docs (series or list): 토큰화 된 문서가 들어있는 list
    Returns:
        list: Dataframe
    """
    
    # word_counts : 말뭉치에서 단어의 개수입니다.
    word_counts = Counter()

    # word_in_docs : 단어가 존재하는 문서의 빈도입니다. 해당 단어가 한 번 이상 존재하면 +1
    word_in_docs = Counter()

    # total_docs : 전체 문서의 개수입니다.
    total_docs = len(docs)

    for doc in docs:
        word_counts.update(doc)
        word_in_docs.update(set(doc))

    temp = zip(word_counts.keys(), word_counts.values())

    wc = pd.DataFrame(temp, columns = ['word', 'count'])

    # rank 열에 단어 빈도 순으로 순위를 저장합니다.
    # method='first': 같은 값의 경우 먼저나온 요소를 상위에 배치합니다.
    wc['rank'] = wc['count'].rank(method='first', ascending=False).astype(int)
    total = wc['count'].sum()

    # percent 열에는 말뭉치 내 단어의 비율을 계산합니다.
    wc['percent'] = wc['count'].apply(lambda x: x / total)

    wc = wc.sort_values(by='rank')

    # cul_percent 열에는 누적 비율을 저장합니다.
    wc['cul_percent'] = wc['percent'].cumsum()

    temp2 = zip(word_in_docs.keys(), word_in_docs.values())
    ac = pd.DataFrame(temp2, columns=['word', 'word_in_docs'])
    wc = ac.merge(wc, on='word')
    
    # word_in_docs_percent 열에는 전체 문서 중 해당 단어가 존재하는 문서의 비율을 저장합니다.
    wc['word_in_docs_percent'] = wc['word_in_docs'].apply(lambda x: x / total_docs)

    return wc.sort_values(by='rank')

In [43]:
# 토큰화된 각 리뷰 리스트를 카운터 객체에 업데이트 합니다. 
df_0['token'].apply(lambda x: word_counts.update(x))

# 가장 많이 존재하는 단어 순으로 10개를 나열합니다
word_counts.most_common(10)



[('and', 43728),
 ('', 42986),
 ('to', 25388),
 ('the', 21076),
 ('of', 17678),
 ('data', 14850),
 ('in', 13538),
 ('a', 12872),
 ('with', 11454),
 ('for', 8264)]

In [44]:
wc = word_count(df_0['token'])
wc.head()

,word,word_in_docs,count,rank,percent,cul_percent,word_in_docs_percent
215,and,755,21864,1,0.053487,0.053487,0.997358
0,,751,21493,2,0.052580,0.106067,0.992074
65,to,754,12694,3,0.031054,0.137121,0.996037
205,the,745,10538,4,0.025780,0.162901,0.984148
3,of,745,8839,5,0.021623,0.184524,0.984148


In [35]:
import seaborn as sns

wc_top10 = wc[wc['rank']<=10]['cul_percent']
wc_top10

215    0.053487
0      0.106067
65     0.137121
205    0.162901
3      0.184524
140    0.202689
94     0.219248
159    0.234993
97     0.249003
478    0.259111
Name: cul_percent, dtype: float64

### 4) 확장된 불용어 사전을 사용해 토큰을 정제합니다.


- **문항 4) 기본 불용어 사전에 두 단어(`"data", "work"`)를 추가하는 코드를 사용해주세요.**
- **문항 5) 불용어를 제거하고 난 뒤 토큰 순위 10개의 단어를 입력하세요.**

In [37]:
print(nlp.Defaults.stop_words)
print(type(nlp.Defaults.stop_words))

{'who', 'afterwards', 'n‘t', 'doing', 'will', 'around', 'anywhere', 'after', 'though', "'s", 'its', 'anything', 'say', 're', 'across', 'be', 'fifty', 'front', 'i', 'but', 'us', 'twenty', 'only', 'all', 'in', 'ten', 'hence', 'together', 'more', 'could', 'them', 'you', 'one', 'however', 'when', 'every', 'into', 'very', 'being', 'seemed', 'done', 'fifteen', 'anyone', 'top', 'ours', 'unless', '‘d', 'him', 'up', '’d', 'beforehand', 'really', 'amongst', 'those', 'used', 'toward', 'ourselves', 'back', 'it', 'along', 'before', 'move', 'four', 'must', 'go', 'keep', 'anyway', 'seems', 'per', 'call', 'other', '‘ll', 'same', 'sixty', 'forty', 'above', 'none', 'several', 'yourself', 'of', 'no', 'meanwhile', 'thereby', 'nothing', 'whom', 'twelve', 'may', 'as', 'serious', 'some', 'each', 'had', 'also', 'empty', 'hereupon', 'hereafter', 'thence', 'former', 'among', 'these', 'their', 'side', 'less', 'everywhere', 'six', 'due', 'to', 'within', 'off', 'anyhow', 'enough', '‘m', 'about', 'between', 'ca', '

In [47]:
STOP_WORDS = nlp.Defaults.stop_words.union(['data','work'])

tokens = []

for doc in tokenizer.pipe(df_0['description']):
  doc_tokens = []

  for token in doc:
    if token.text.lower() not in STOP_WORDS:
      doc_tokens.append(token.text.lower())

    tokens.append(doc_tokens)

df['tokens'] = tokens

ValueError: ignored

### 5) Lemmatization 사용 효과를 분석해 봅니다.



- **문항 6) Lemmatization을 진행한 뒤 상위 10개 단어를 입력하세요.**

In [ ]:
### 이곳에서 과제를 진행해 주세요 ### 

## 2. 유사한 문서 찾기

### 1) `TfidfVectorizer`를 이용해 각 문서들을 벡터화 한 후 KNN 모델을 만들고, <br/> 내가 원하는 `job description`을 질의해 가장 가까운 검색 결과들을 가져오고 분석합니다.

- **문항 9) 88번 index의 `job description`와 5개의 가장 유사한 `job description`이 있는 index를 입력하세요.**
    - 답은 88번 인덱스를 포함합니다.
    - `max_features = 3000` 으로 설정합니다.
    - [88, 90, 91, 93, 94] 형태로 답을 입력해주세요

## 3. TF-IDF 이용한 텍스트 분류 진행하기

TF-IDF를 이용해 문장 혹은 문서를 벡터화한 경우, 이 벡터값을 이용해 문서 분류 태스크를 진행할 수 있습니다. 

현재 다루고 있는 데이터셋에는 label이 존재하지 않으므로, title 컬럼에 "Senior"가 있는지 없는지 여부를 통해 Senior 직무 여부를 분류하는 작업을 진행해보겠습니다.

### 1) title 컬럼에 "Senior" 문자열이 있으면 1, 없으면 0인 "Senior"라는 새로운 컬럼을 생성해주세요.

문항 7) 새롭게 만든 Senior 컬럼에서 값이 1인 (Senior O) 데이터의 개수는?

In [ ]:
df['senior'] = df['title'].apply()

문항 8) sklearn의 `train_test_split`을 통해 train 데이터와 valid 데이터로 나눈 후, `sklearn`의 `DecisionTreeClassifier`를 이용해 분류를 진행해주세요. 

단, x값은 위에서 학습한 dtm_tfidf를 그대로 이용해주세요. train_test_split과 DecisionTreeClassifier의 random_state을 42로 고정하고, test_size는 0.1로 설정해주세요.

학습을 완료한 후, test 데이터에 대한 예측을 진행하고 label 1에 대한 precision과 recall 값을 적어주세요

In [ ]:
### 이곳에서 과제를 진행해 주세요 ### 